# MODULE 6 - Challenge

## PART 2: Have Customers Narrow Travel Searches Based on Temp & Precipitation

In [1]:
import pandas as pd
import requests
import gmaps
from config import g_key
import csv


In [2]:
# 2. Import the "WeatherPy_vacation.csv" file from PART 1 as a new DataFrame.
     # Instructions are off, should be "WeatherPy_challenge.csv" file.

city_data_to_load = "challenge_data/WeatherPy_challenge.csv"

city_data_df = pd.read_csv(city_data_to_load, index_col=0)  # Added "index_col=0" to remove unnamed column.

city_data_df.head()


,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3hrs),Snow inches (last 3hrs)
0,Armenokhorion,GR,2020-01-05 21:00:00,40.8026,21.4609,33.80,75,100,10.47,overcast clouds,0.00,0.0
1,Sandwick,GB,2020-01-05 21:00:00,54.5691,-2.8936,46.85,97,100,12.06,overcast clouds,0.00,0.0
2,Fereydun Kenar,IR,2020-01-05 21:00:00,36.6864,52.5225,45.84,92,58,3.49,broken clouds,0.00,0.0
3,Guerrero,AR,2020-01-05 21:00:00,-24.1859,-65.4469,80.56,86,78,1.63,light rain,1.56,0.0
4,Acapulco,MX,2020-01-05 21:00:00,16.8603,-99.8769,83.88,63,80,7.18,broken clouds,0.00,0.0


In [3]:
city_data_df.dtypes


City                        object
Country                     object
Date                        object
Lat                        float64
Lng                        float64
Max Temp                   float64
Humidity                     int64
Cloudiness                   int64
Wind Speed                 float64
Current Description         object
Rain inches (last 3hrs)    float64
Snow inches (last 3hrs)    float64
dtype: object

In [4]:
# 3. Filter the DataFrame for min & max temp preferences, and if rain or snow accumulation is 0 inches.

# Prompt customer for the min temp preference.
min_temp = float(input("What is your minimum temperature preference? "))

# Prompt customer for the max temp preference.
max_temp = float(input("What is your maximum temperature preference? "))

# Prompt customer to answer "yes" or "no" to rainfall.
rain_prefer = input("Do you want it to be raining? (yes/no) ")

# Prompt customer to answer "yes" or "no" to snowfall.
snow_prefer = input("Do you want it to be snowing? (yes/no) ")

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) ]

if rain_prefer.lower() == "yes":
    preferred_cities_df = preferred_cities_df.loc[(city_data_df["Rain inches (last 3hrs)"] > 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(city_data_df["Rain inches (last 3hrs)"] == 0)]

if snow_prefer.lower() == "yes":
    preferred_cities_df = preferred_cities_df.loc[(city_data_df["Snow inches (last 3hrs)"] > 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(city_data_df["Snow inches (last 3hrs)"] == 0)]

preferred_cities_df.head()


What is your minimum temperature preference? 0
What is your maximum temperature preference? 110
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3hrs),Snow inches (last 3hrs)
0,Armenokhorion,GR,2020-01-05 21:00:00,40.8026,21.4609,33.80,75,100,10.47,overcast clouds,0.0,0.0
1,Sandwick,GB,2020-01-05 21:00:00,54.5691,-2.8936,46.85,97,100,12.06,overcast clouds,0.0,0.0
2,Fereydun Kenar,IR,2020-01-05 21:00:00,36.6864,52.5225,45.84,92,58,3.49,broken clouds,0.0,0.0
4,Acapulco,MX,2020-01-05 21:00:00,16.8603,-99.8769,83.88,63,80,7.18,broken clouds,0.0,0.0
5,Meulaboh,ID,2020-01-05 21:00:00,4.1363,96.1285,80.40,81,0,6.78,clear sky,0.0,0.0


In [5]:
# 4. The following entries are to add cities to a marker layer map with pop-up markers that include hotel name, 
# city, country, current weather description, and max temperature.

In [6]:
# Configure gmaps to use my Google API key.

gmaps.configure(api_key=g_key)


In [7]:
# Create the hotel_df to store hotel names under new column "Hotel Name" & pull other colums using copy().

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()

hotel_df["Hotel Name"] = ""

hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Armenokhorion,GR,33.80,overcast clouds,40.8026,21.4609,
1,Sandwick,GB,46.85,overcast clouds,54.5691,-2.8936,
2,Fereydun Kenar,IR,45.84,broken clouds,36.6864,52.5225,
4,Acapulco,MX,83.88,broken clouds,16.8603,-99.8769,
5,Meulaboh,ID,80.40,clear sky,4.1363,96.1285,
7,Yara,CU,85.30,overcast clouds,20.2740,-76.9533,
8,Ribeira Brava,PT,64.49,clear sky,32.6500,-17.0667,
10,Colon,PH,78.85,overcast clouds,10.2172,123.7548,
11,Christchurch,NZ,77.68,clear sky,-43.5310,172.6366,
12,Gerga,RU,44.44,clear sky,42.3465,47.9633,


In [8]:
# Set parameters to search for hotels using an API key, lat & lng, 5K meter radius.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [9]:
# Iterate through hotel_df using the iterrows() function and a for loop.

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Armenokhorion,GR,33.80,overcast clouds,40.8026,21.4609,"PLEIADES, ROOMS"
1,Sandwick,GB,46.85,overcast clouds,54.5691,-2.8936,Macdonald Leeming House
2,Fereydun Kenar,IR,45.84,broken clouds,36.6864,52.5225,Tourist Hotel
4,Acapulco,MX,83.88,broken clouds,16.8603,-99.8769,Crowne Plaza Acapulco
5,Meulaboh,ID,80.40,clear sky,4.1363,96.1285,Kost Pria


In [11]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


In [12]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [13]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# 5. Save and upload the new DataFrame as "WeatherPy_vacation.csv"

output_data_file = "challenge_data/WeatherPy_vacation.csv"

city_data_df.to_csv(output_data_file)


In [15]:
# 6. Save and upload the new marker layer map as "WeatherPy_vacation_map.png"

# Used the "download" button to save map as .png, but opened markers do not get saved.
